# Collecting and preparing data before analysis
How to start? I have several .gpx files, one for each run, over several years and I want to generate some visualizations based on those data. To do so I'm using the pandas library.

Before we start, the only few things I know about those gps paths are the followings:
+ one path per week *when I have the data*
+ it starts from different locations, but each path is a loop *ie the path comes back to its origin*
+ it almost always reaches the same middle point *ie the same gps location*

## Collect data
It is the part of the project that requires a huge amount of time.

I ended up gathering .gpx files, one for each run. Here I just renamed each file according to the date the run was done.

### Check data files to be processed

In [7]:
!ls ../data/gpx/RunRite*gpx | wc -l

     271


In [8]:
!ls ../data/csv/RunRite*cleaned.csv | wc -l

zsh:1: no matches found: ../data/csv/RunRite*cleaned.csv
       0


In [9]:
!rm ../data/csv/RunRite*csv

zsh:1: no matches found: ../data/csv/RunRite*csv


In [10]:
!ls ../data/csv/RunRite*csv | wc -l

zsh:1: no matches found: ../data/csv/RunRite*csv
       0


### Import a few modules

In [1]:
import sys
import numpy as np
import pandas as pd
import gpxpy
import gpxpy.gpx
import matplotlib.pyplot as plt
import geopy.distance
import glob
import os
import importlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from datetime import datetime
import time

below I'm loading my module.

In [11]:
sys.path.append("../my_modules")
import toolToReadGPX as ttrgpx

path_data     = "../data/"
path_data_csv = "../data/csv/"
run_club_name = "RunRite" 

# Rename files

I want all files to be as follows **RunRite_year_month_day.gpx**.

I will:
+ list the files
+ lower case all the files
+ re-write **RunRite**

In [12]:
# list and rename the files
ll = glob.glob(path_data+"gpx/"+run_club_name+"*.gpx")
ll.sort()
flag_rename = "False"
print(len(ll), ll[-4:])

#for c, l in enumerate(ll):
#    head_tail = os.path.split(ll[c])
#    dst_tail  = head_tail[1].lower()
#    dst_head_tail = head_tail[0]+"/"+run_club_name+"_"+dst_tail[-14:]
#    if flag_rename == True:
#        if dst_tail[-5] == "b": # in case we have two runs
#            dst_head_tail = head_tail[0]+"/"+run_club_name+"_"+dst_tail[-15:]
#            os.rename(ll[c], dst_head_tail) 
#        else:
#            os.rename(ll[c], dst_head_tail)

271 ['../data/gpx/RunRite_2024_10_31.gpx', '../data/gpx/RunRite_2024_11_07.gpx', '../data/gpx/RunRite_2024_11_14.gpx', '../data/gpx/RunRite_2024_11_22.gpx']


In [53]:
# list all available gpxfile
list_all_files = glob.glob(path_data+"gpx/"+run_club_name+"*.gpx")
list_all_files.sort()

# load last gpx file
single_file_name = list_all_files[-1]
single_file_df = ttrgpx.fun_gpx2pd(single_file_name)

# reduce file size
single_file_resample_df = ttrgpx.fun_DownSample_gpx(single_file_df, number_of_sample = 200)

# create new dataFrame
all_info_df = pd.DataFrame(columns = ["time","cumulative_distance","latitude","longitude","elevation"])
print(single_file_df['cumulative_distance'].iloc[-1])
list_latitude = single_file_df['latitude'].values.tolist()
all_info_df["cumulative_distance"] = single_file_df['cumulative_distance'].values
all_info_df["latitude"] = single_file_df['latitude'].values.tolist()


dd = [Point(xyz) for xyz in zip(single_file_df['latitude'].values.tolist(), 
                                single_file_df['longitude'].values.tolist(),
                                single_file_df['elevation'].values.tolist())] 


dd

12280.795054335926


NameError: name 'Point' is not defined

In [47]:
all_info_df

,time,cumulative_distance,latitude,longitude,elevation
0,NaN,0.000000,45.489910,NaN,NaN
1,NaN,0.000000,45.489910,NaN,NaN
2,NaN,0.000000,45.489910,NaN,NaN
3,NaN,0.000000,45.489910,NaN,NaN
4,NaN,0.000000,45.489910,NaN,NaN
...,...,...,...,...,...
5302,NaN,12280.795054,45.489715,NaN,NaN
5303,NaN,12280.795054,45.489715,NaN,NaN
5304,NaN,12280.795054,45.489715,NaN,NaN
5305,NaN,12280.795054,45.489715,NaN,NaN


In [9]:
importlib.reload(ttrgpx)

# Select gpx file
list_all_files = glob.glob(path_data+"/gpx/"+run_club_name+"*.gpx")
list_all_files.sort()
print("There is {0:1.0f} files to process.".format(len(list_all_files)))

# convert list of files to a list of dataFrame
#list_all_files_df = ttrgpx.fun_listPath_gpx2pd(list_all_files)
#print("--- %s seconds for loopy loop ---" % (time.time() - start_time))

# get number of run
#nb_run = len(list_all_files_df)

#new_list = [expression for member in iterable]
list_all_files_as_df = [ttrgpx.fun_gpx2pd(single_gpx) for single_gpx in list_all_files]

# get number of run
nb_run = len(list_all_files_as_df)

There is 271 files to process.


In [10]:
# get average distance for this year
list_cumulative_distance = [x.iloc[-1,4] for x in list_all_files_as_df]
average_run_distance     = sum(list_cumulative_distance) / (1000*len(list_cumulative_distance))

print("Average distance per run for all the gpx paths listed: {0:1.2f}km.".format(average_run_distance))

Average distance per run for all the gpx paths listed: 9.82km.


Check how one gpx path looks like:

# Reduce data size and data as csv

Here I will reduce the data size as I don't need so many points (eg here more than 1000) and will reduce the length to **x** points per gpx points.

And I will save the downsample path as csv files.

## Reduce data size

In [11]:
!ls ../data/gpx/Run*

../data/gpx/RunRite_2017_09_14.gpx  ../data/gpx/RunRite_2021_08_05.gpx
../data/gpx/RunRite_2017_09_21.gpx  ../data/gpx/RunRite_2021_08_12.gpx
../data/gpx/RunRite_2017_09_28.gpx  ../data/gpx/RunRite_2021_08_26.gpx
../data/gpx/RunRite_2017_10_05.gpx  ../data/gpx/RunRite_2021_09_01.gpx
../data/gpx/RunRite_2017_10_12.gpx  ../data/gpx/RunRite_2021_09_09.gpx
../data/gpx/RunRite_2017_10_19.gpx  ../data/gpx/RunRite_2021_09_16.gpx
../data/gpx/RunRite_2017_10_26.gpx  ../data/gpx/RunRite_2021_09_23.gpx
../data/gpx/RunRite_2017_11_02.gpx  ../data/gpx/RunRite_2021_09_30.gpx
../data/gpx/RunRite_2017_11_09.gpx  ../data/gpx/RunRite_2021_10_07.gpx
../data/gpx/RunRite_2017_11_16.gpx  ../data/gpx/RunRite_2021_10_14.gpx
../data/gpx/RunRite_2017_11_23.gpx  ../data/gpx/RunRite_2021_10_21.gpx
../data/gpx/RunRite_2017_11_30.gpx  ../data/gpx/RunRite_2021_10_28.gpx
../data/gpx/RunRite_2017_12_14.gpx  ../data/gpx/RunRite_2021_11_11.gpx
../data/gpx/RunRite_2017_12_21.gpx  ../data/gpx/RunRite_2021_11_18.gpx
../dat

In [12]:
importlib.reload(ttrgpx)

# reduce the size
list_all_files_ReSample_as_df = [ttrgpx.fun_DownSample_gpx(x, number_of_sample = 200) for x in list_all_files_as_df]

# check one gpx path
list_all_files_ReSample_as_df[0].describe()

,latitude,longitude,elevation,distance,cumulative_distance
count,200.000000,200.000000,200.00000,200.000000,200.000000
mean,45.468915,-73.624723,73.28500,2.653835,4927.159937
std,0.005679,0.008367,8.47128,1.055967,2744.096143
min,45.458731,-73.640191,60.00000,0.000000,0.000000
25%,45.464529,-73.632023,66.00000,2.362598,2647.566083
50%,45.469368,-73.622687,71.00000,2.771953,5010.515600
75%,45.473876,-73.617976,78.00000,3.158386,7267.507413
max,45.478080,-73.610549,93.00000,7.434511,9549.687209


In [11]:
head_tail = os.path.split(ll[-1])
print(head_tail[1][0:-4])

RunRite_2024_10_24


# Save as csv

In [13]:
head_tail = os.path.split(ll[c])
print(head_tail)

('../data/gpx', 'RunRite_2024_11_22.gpx')


In [14]:
# get list of gpx file nale
list_gpx_file = glob.glob(path_data+"gpx/"+run_club_name+"*.gpx")
list_gpx_file.sort()

# create list of csv file name
list_csv_file_name = [path_data_csv+os.path.split(x)[1][0:-4]+"_downSample.csv" for x in list_gpx_file]

# export DataFrame as csv file
for c,df_ in enumerate(list_all_files_ReSample_as_df):
    df_.to_csv(list_csv_file_name[c], index=False)
    

In [15]:
!ls ../data/csv/RunRite*.csv | wc -l

     271


In [16]:
!ls ../data/csv/RunRite*.csv

../data/csv/RunRite_2017_09_14_downSample.csv
../data/csv/RunRite_2017_09_21_downSample.csv
../data/csv/RunRite_2017_09_28_downSample.csv
../data/csv/RunRite_2017_10_05_downSample.csv
../data/csv/RunRite_2017_10_12_downSample.csv
../data/csv/RunRite_2017_10_19_downSample.csv
../data/csv/RunRite_2017_10_26_downSample.csv
../data/csv/RunRite_2017_11_02_downSample.csv
../data/csv/RunRite_2017_11_09_downSample.csv
../data/csv/RunRite_2017_11_16_downSample.csv
../data/csv/RunRite_2017_11_23_downSample.csv
../data/csv/RunRite_2017_11_30_downSample.csv
../data/csv/RunRite_2017_12_14_downSample.csv
../data/csv/RunRite_2017_12_21_downSample.csv
../data/csv/RunRite_2018_01_04_downSample.csv
../data/csv/RunRite_2018_01_11_downSample.csv
../data/csv/RunRite_2018_01_18_downSample.csv
../data/csv/RunRite_2018_02_01_downSample.csv
../data/csv/RunRite_2018_02_08_downSample.csv
../data/csv/RunRite_2018_02_15_downSample.csv
../data/csv/RunRite_2018_02_22_downSample.csv
../data/csv/RunRite_2018_03_01_dow

Check that the csv files has been created.

In [17]:
list_all_files_ReSample_as_df[-1].describe()

,latitude,longitude,elevation,distance,cumulative_distance
count,200.000000,200.000000,200.000000,200.000000,200.000000
mean,45.490638,-73.587808,97.152000,2.247500,6342.784696
std,0.005754,0.013940,66.188178,1.385497,3449.932350
min,45.477437,-73.610784,22.600000,0.000000,0.000000
25%,45.487252,-73.601087,28.800000,1.201154,3577.073367
50%,45.491568,-73.588637,76.800000,2.700610,6123.283889
75%,45.494384,-73.575416,158.400000,3.123018,9274.111615
max,45.501194,-73.564390,202.400000,5.527080,12280.795054


In [17]:
print(list_all_files_ReSample_as_df[0]["cumulative_distance"].iloc[-1])
print(list_all_files_ReSample_as_df[-1]["cumulative_distance"].iloc[-1])

9549.68720853391
13628.262546102178


### Check we have the file in csv

In [18]:
# load one file
new_df = pd.read_csv(path_data_csv+"RunRite_2024_09_19b_downSample.csv")

# to check if it is the same as the list of Dataframe
new_df["cumulative_distance"].iloc[-1] - list_all_files_ReSample_as_df[-1]["cumulative_distance"].iloc[-1]

-3995.789929966704

# Now the datavisulization can start

Before let's have a look at the data I have, how they are organized.

In folders I have:
+ *../data/gpx/* the .gpx files for each run, each of them name **RunRite_year_month_day.gpx**
+ *../data/csv/* the .csv files for each run in reduced size comparing to the .gpx files , each of them name **RunRite_year_month_day_downSample.csv**

# Re-load the data

I want to have:
+ a list of all file names
+ a list of panda DataFrame where each element is for a single run

In [19]:
# list of file names
path_csv_files = "../data/csv/"
list_csv_files  = glob.glob(path_csv_files+"*.csv")
list_csv_files.sort()

# list of panda DataFrame
#list_run_df = []
#for c, f in enumerate(list_csv_files):
#    list_run_df.append(pd.read_csv(f))

list_run_df2 = [pd.read_csv(x) for x in list_csv_files]

# Add extra data cleaning

I need to:
+ clean data to keep only the gps trace of the run
+ to remove all point before and after arrival

# Adding missing data

Sometimes I don't have the data, so I get them from other runners, as we don't use all the same device to record our run we don't have always the same extra information. Therefore I skipped adding the _time_ added to each (longitude, latitude) point.